In [1]:
#import the library
import requests
from time import sleep 
import re
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# define header to access google scholar website
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.placeholder Safari/537.36'}

In [3]:
# this function for the getting inforamtion of the web page
import random
import copy

ips = list(range(100))

def get_paperinfo(paper_url):

  #download the page
  hd = copy.deepcopy(headers)
  hd['user-agent']=hd['user-agent'].replace("placeholder",str(random.choice(ips)))
  response=requests.get(url,headers=hd)

  # check successful response
  if response.status_code != 200:
    print('Status code:', response.status_code)
    raise Exception('Failed to fetch web page ')

  #parse using beautiful soup
  paper_doc = BeautifulSoup(response.text,'html.parser')

  return paper_doc

In [4]:
# this function for the extracting information of the tags
def get_tags(doc):
  paper_tag = doc.select('[data-lid]')
  cite_tag = doc.find_all(lambda tag: tag.name == 'div' and tag.get('class') == ['gs_fl'])
  link_tag = doc.find_all('h3',{"class" : "gs_rt"})
  author_tag = doc.find_all("div", {"class": "gs_a"})

  return paper_tag,cite_tag,link_tag,author_tag

In [5]:
# it will return the title of the paper
def get_papertitle(paper_tag):
  
  paper_names = []
  
  for tag in paper_tag:
    paper_names.append(tag.select('h3')[0].get_text())

  return paper_names

In [6]:
# it will return the number of citation of the paper
def get_citecount(cite_tag):
  cite_count = []
  for i in cite_tag:
    cite = i.text
    if i is None or cite is None:  # if paper has no citatation then consider 0
      cite_count.append(0)
    else:
      tmp = re.search(r'\d+', cite) # its handle the None type object error and re use to remove the string " cited by " and return only integer value
      if tmp is None :
        cite_count.append(0)
      else :
        cite_count.append(int(tmp.group()))

  return cite_count

In [7]:
# function for the getting link information
def get_link(link_tag):

  links = []

  for i in range(len(link_tag)) :
#     print(link_tag[i].a)
#     print()
    if link_tag[i].a:
        links.append(link_tag[i].a['href'])
    else:
        links.append("unavailable")

  return links 

In [8]:
# function for the getting autho , year and publication information
def get_author_year_publi_info(authors_tag):
  years = []
  publication = []
  authors = []
  for i in range(len(authors_tag)):
      authortag_text = (authors_tag[i].text).split()
      if not authors_tag[i] or not authors_tag[i].text:
            continue
      try:
        year = int(re.search(r'\d+', authors_tag[i].text).group())
      except Exception as e:
        year = 1000

        
      years.append(year)
      publication.append(authortag_text[-1])
      author = authortag_text[0] + ' ' + re.sub(',','', authortag_text[1])
      authors.append(author)
  
  return years , publication, authors

In [9]:
# creating final repository
paper_repos_dict = {
                    'Paper Title' : [],
                    'Year' : [],
                    'Author' : [],
                    'Citation' : [],
                    'Publication' : [],
                    'Url of paper' : [] }

# adding information in repository
def add_in_paper_repo(papername,year,author,cite,publi,link):
  paper_repos_dict['Paper Title'].extend(papername)
  paper_repos_dict['Year'].extend(year)
  paper_repos_dict['Author'].extend(author)
  paper_repos_dict['Citation'].extend(cite)
  paper_repos_dict['Publication'].extend(publi)
  paper_repos_dict['Url of paper'].extend(link)

  return pd.DataFrame(paper_repos_dict)

In [12]:
import random

sleeps = list(range(5,15))

for i in range (0,400,10):

  # get url for the each page
  url = f"https://scholar.google.com/scholar?start={i}&q=neurobiology+of+Adult+ADHD&hl=en&as_sdt=0,5"
  print(url)

  # function for the get content of each page
  doc = get_paperinfo(url)

  # function for the collecting tags
  paper_tag,cite_tag,link_tag,author_tag = get_tags(doc)
  
  # paper title from each page
  papername = get_papertitle(paper_tag)

  # year , author , publication of the paper
  year , publication , author = get_author_year_publi_info(author_tag)

  # cite count of the paper 
  cite = get_citecount(cite_tag)

  # url of the paper
  link = get_link(link_tag)

  # add in paper repo dict
  final = add_in_paper_repo(papername,year,author,cite,publication,link)
  
  #use sleep to avoid status code 429
    
  sleep(random.choice(sleeps))

https://scholar.google.com/scholar?start=0&q=neurobiology+of+Adult+ADHD&hl=en&as_sdt=0,5
https://scholar.google.com/scholar?start=10&q=neurobiology+of+Adult+ADHD&hl=en&as_sdt=0,5
https://scholar.google.com/scholar?start=20&q=neurobiology+of+Adult+ADHD&hl=en&as_sdt=0,5
https://scholar.google.com/scholar?start=30&q=neurobiology+of+Adult+ADHD&hl=en&as_sdt=0,5
https://scholar.google.com/scholar?start=40&q=neurobiology+of+Adult+ADHD&hl=en&as_sdt=0,5
https://scholar.google.com/scholar?start=50&q=neurobiology+of+Adult+ADHD&hl=en&as_sdt=0,5
https://scholar.google.com/scholar?start=60&q=neurobiology+of+Adult+ADHD&hl=en&as_sdt=0,5
https://scholar.google.com/scholar?start=70&q=neurobiology+of+Adult+ADHD&hl=en&as_sdt=0,5
https://scholar.google.com/scholar?start=80&q=neurobiology+of+Adult+ADHD&hl=en&as_sdt=0,5
https://scholar.google.com/scholar?start=90&q=neurobiology+of+Adult+ADHD&hl=en&as_sdt=0,5
https://scholar.google.com/scholar?start=100&q=neurobiology+of+Adult+ADHD&hl=en&as_sdt=0,5
https://sc

In [13]:
len(final)

710

In [ ]:
final[:100]

In [ ]:
final.to_csv('ADHD_in_adults.csv', sep=',', index=False,header=True)